참고자료: https://www.dinolabs.ai/400

In [ ]:
!pip install transformers==4.28.0 
!git clone https://github.com/phoo8651/KoHateSpeech.git

import pandas as pd
import matplotlib.pyplot as plt

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

#GPU설정
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print("device:", device)

df = pd.read_csv("/content/KoHateSpeech/Hate-Data.csv", sep=",")
df.head()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 50.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 65.8 MB/s eta 0:00:00
Cloning into 'KoHateSpeech'...
remote: Enumerating objects: 43, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 43 (delta 16), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (43/43), 2.60 MiB | 3.18 MiB/s, done.
device: cuda:0


,content,label
0,너무 예뻐요,1
1,조선일보 출신 정치인 조씨가 누구냐?,0
2,gov만 들어가면 무조건 다 공식싸이튼줄 착각하는 미드보고 미국배운 조선놈ㅋㅋ. 어...,0
3,서울에 왜 핀란드가 있어야 하는가,1
4,씨발 문재앙 때문이냐? 왜 점점 사회가 공산주의화 되가냐?니들 보고 용접공 하려면 ...,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16606 entries, 0 to 16605
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   content  16606 non-null  object
 1   label    16606 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 259.6+ KB


In [ ]:
null_idx = df[df.label.isnull()].index

df.loc[null_idx, "label"] = df.loc[null_idx, "label"].apply(lambda x: x[:-1])
df.loc[null_idx, "content"] = df.loc[null_idx, "content"].apply(lambda x: x[:-2])

df = df.astype({"label": "int"})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16606 entries, 0 to 16605
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   content  16606 non-null  object
 1   label    16606 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 259.6+ KB


In [ ]:
train_data = df.sample(frac=0.8, random_state=42)
test_data = df.drop(train_data.index)

#데이터셋 갯수 확인
print('중복 제거 전 학습 데이터 : {}'.format(len(train_data)))
print('중복 제거 전 테스트 데이터 : {}'.format(len(test_data)))

train_data.drop_duplicates(subset=["content"], inplace=True)
test_data.drop_duplicates(subset=["content"], inplace=True)

#데이터셋 갯수 확인
print('중복 제거 후 학습 데이터 : {}'.format(len(train_data)))
print('중복 제거 후 테스트 데이터 : {}'.format(len(test_data)))

중복 제거 전 학습 데이터 : 13285
중복 제거 전 테스트 데이터 : 3321
중복 제거 후 학습 데이터 : 13285
중복 제거 후 테스트 데이터 : 3321


In [ ]:
MODEL_NAME = "beomi/KcELECTRA-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenized_train_sentences = tokenizer(list(train_data["content"]), return_tensors="pt", max_length=128, padding=True, truncation=True, add_special_tokens=True)
tokenized_test_sentences = tokenizer(list(test_data["content"]), return_tensors="pt", max_length=128, padding=True, truncation=True, add_special_tokens=True) 

In [ ]:
class CurseDataset(torch.utils.data.Dataset):
  def __init__(self, encodings, labels):
    self.encodings = encodings
    self.labels = labels
  
  def __getitem__(self, idx):
    item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    item["labels"] = torch.tensor(self.labels[idx])
    return item

  def __len__(self):
    return len(self.labels)

In [ ]:
train_label = train_data["label"].values
test_label = test_data["label"].values

train_dataset = CurseDataset(tokenized_train_sentences, train_label)
test_dataset = CurseDataset(tokenized_test_sentences, test_label)

model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)
model.to(device)

Some weights of the model checkpoint at beomi/KcELECTRA-base were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base and are newly initialized: ['classifier.dense.weight', 'classifier.

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=3)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): L

In [ ]:
training_args = TrainingArguments(
    output_dir="./",
    num_train_epochs=12,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=64,
    logging_dir="./logs",
    logging_steps=500,
    save_total_limit=2
)

In [ ]:
def compute_metrics(pred):
  labels = pred.label_ids,
  preds = pred.predictions.argmax(-1)
  precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary")
  acc = accurary_score(labels, preds)
  return {
      'accuray': acc,
      'f1': f1,
      'precision': precision,
      'recall': recall
  }

In [ ]:
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = test_dataset,
    compute_metrics = compute_metrics
)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
<ipython-input-6-106b88d57b26>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Step,Training Loss
500,0.429500
1000,0.353600
1500,0.326100
2000,0.269400
2500,0.239300
3000,0.249900
3500,0.201400
4000,0.169600
4500,0.186600
5000,0.161700


<ipython-input-6-106b88d57b26>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-6-106b88d57b26>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-6-106b88d57b26>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-6-106b88d57b26>:7: UserWarning: To copy construct from a tens

TrainOutput(global_step=19932, training_loss=0.1112581238123683, metrics={'train_runtime': 4276.5099, 'train_samples_per_second': 37.278, 'train_steps_per_second': 4.661, 'total_flos': 1.04862911113728e+16, 'train_loss': 0.1112581238123683, 'epoch': 12.0})

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
folder_path = "/content/drive/MyDrive/pyTorch/HateSpeech"
model.save_pretrained(folder_path)

Mounted at /content/drive


In [ ]:
!pip install transformers==4.28.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 74.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 106.1 MB/s eta 0:00:00
Cloning into 'korean-hate-speech'...
remote: Enumerating objects: 112, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 112 (delta 4), reused 0 (delta 0), pack-reused 103
Receiving objects: 100% (112/112), 93.18 MiB | 6.79 MiB/s, done.
Resolving deltas: 100% (48/48), done.
Updating files: 100% (20/20), done.


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from google.colab import drive
import pandas as pd

drive.mount('/content/drive')
folder_path = "/content/drive/MyDrive/pyTorch/HateSpeech"

MODEL_NAME = "beomi/KcELECTRA-base"
model = AutoModelForSequenceClassification.from_pretrained(folder_path)
tokenizer = AutoTokenizer.from_pretrained("beomi/KcELECTRA-base")
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

def sentence_predict(sent):
  model.eval()
  tokenized_sent = tokenizer(
      sent,
      return_tensors="pt",
      truncation=True,
      add_special_tokens=True,
      max_length=128
  )

  tokenized_sent["input_ids"] = tokenized_sent["input_ids"].to(device)
  tokenized_sent["attention_mask"] = tokenized_sent["attention_mask"].to(device)
  tokenized_sent["token_type_ids"] = tokenized_sent["token_type_ids"].to(device)

  model.to(device)

  with torch.no_grad():
    outputs = model(**tokenized_sent)

  logits = outputs.logits
  logits = logits.detach().cpu()
  result = logits.argmax(-1)
  if result == 0:
    return 0
  else:
    return 1


Mounted at /content/drive


In [ ]:
text = "세상은 아직 아름답다"
print(sentence_predict(text))

1
